In [1]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import logging
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import cv2
from pynotate import Project

from dnafiber.inference import _get_model
from dnafiber.deployment import run_one_file
from dnafiber.ui.utils import get_image_cacheless
from tqdm.auto import tqdm
import numpy as np
from skimage.segmentation import expand_labels
for name, l in logging.root.manager.loggerDict.items():
    if "streamlit" in name:
        l.disabled = True

/home/clement/miniconda3/envs/dnafiber/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/clement/miniconda3/envs/dnafiber/lib/python3.12/site-packages/albumentations/__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.8' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
/home/clement/miniconda3/envs/dnafiber/lib/python3.12/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)
2025-06-30 09:50:34.559 WARNING streamlit.runtime.caching.cache_data_api: No runti

In [2]:
root = "/home/clement/Documents/data/DNAFiber/Yosra/"
all_files = list(Path(root).rglob("*.czi"))
conditions = ['-'.join(f.name.split("-")[:-1]) for f in all_files]
count = {c: 0 for c in set(conditions)}
model = _get_model('unet_se_resnet50_finetuned').cuda()


def extract_fiber_data(df, image: np.ndarray):
    imgs = []
    imgs_with_bbox = []
    segmentation = []

    for i, row in df.iterrows():
        bbox = row["bbox"]
        x, y, w, h = bbox
        seg = row["segmentation"]
        
        seg = expand_labels(seg, distance=2)
        # Compute an offset that doubles the size of the bounding box
        offsetX = int(w * 0.5)
        offsetY = int(h * 0.5)
        # Compute the offset so that the bounding is square
        if offsetX < offsetY:
            offsetX = offsetY
        else:
            offsetY = offsetX


        img = image[y - offsetY : y + h + offsetY, x - offsetX : x + w + offsetX]
        img_with_bbox = img.copy()
        # Ensure img_with_bbox is contiguous in memory for OpenCV compatibility
        img_with_bbox = np.ascontiguousarray(img_with_bbox)
        # Draw the bounding box on the image
        try:
            cv2.rectangle(img_with_bbox, (offsetX, offsetY), (offsetX + w, offsetY + h), (0, 0, 255), 2)
        except Exception as e:
            # print(f"Error drawing rectangle: {e}")
            continue
        
        # Pad the segmentation mask to match the extended bounding box
        padded_seg = np.zeros_like(img_with_bbox[:,:, 0], dtype=np.uint8)
        padded_seg[offsetY:offsetY + h, offsetX:offsetX + w] = seg

        # Resize all the images to a fixed size of 128x128
        img = cv2.resize(img, (128, 128))
        img_with_bbox = cv2.resize(img_with_bbox, (128, 128))
        padded_seg = cv2.resize(padded_seg, (128, 128), interpolation=cv2.INTER_NEAREST_EXACT)
        imgs.append(img)
        imgs_with_bbox.append(img_with_bbox)
        segmentation.append(padded_seg)
        
    return imgs, imgs_with_bbox, segmentation

project = Project("DNAICorrection", 
                  "/home/clement/Documents/data/DNAFiber/DNAICorrection/Input", 
                  "/home/clement/Documents/data/DNAFiber/DNAICorrection/Output",
                  is_segmentation=True, is_classification=True, segmentation_classes=["Red", "Green"],
                  classification_classes=[{'name': 'Quality', 'classes': ['Good', 'Bad']}],)


thumbnails_root = Path("/home/clement/Documents/data/DNAFiber/DNAICorrection/Thumbnails")    
thumbnails_root.mkdir(exist_ok=True, parents=True)
limit_fiber = 50

# with project:
for f in tqdm(all_files):
    condition = '-'.join(f.name.split("-")[:-1])
    if count[condition] >= limit_fiber:
        continue  # Skip if the limit for this condition is reached

    image = get_image_cacheless(f, False)
    df = run_one_file(
        image,
        model=model,
        verbose=False,
        create_thumbnail=False,
        include_bbox=True,
        include_segmentation=True,
    )
    imgs, imgs_with_bbox, segmentation = extract_fiber_data(df, image)

    # Only process up to the remaining allowed fibers for this condition
    remaining = limit_fiber - count[condition]
    imgs = imgs[:remaining]
    imgs_with_bbox = imgs_with_bbox[:remaining]
    segmentation = segmentation[:remaining]

    for i, (img, img_with_bbox, seg) in enumerate(zip(imgs, imgs_with_bbox, segmentation)):
        # project.load_image(
        #     image=np.ascontiguousarray(img_with_bbox),
        #     filename=f"{condition}_{i+count[condition]}.png",
        #     segmentation_masks=[seg==j for j in [1, 2]],
        # )
        cv2.imwrite(str(thumbnails_root / f"{condition}_{i+count[condition]}.png"), img[:,:, ::-1])

    count[condition] += len(imgs)

    


  0%|          | 0/316 [00:00<?, ?it/s]2025-06-30 09:50:36.564 
  command:

    streamlit run /home/clement/miniconda3/envs/dnafiber/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
100%|██████████| 316/316 [06:32<00:00,  1.24s/it]
